# Manchester Property Data Wrangling & Enrichment

This notebook initiates the data preparation process for the Manchester Property Sales Challenge. The focus is on cleaning, transforming, and enriching the dataset to support downstream analysis and dashboard development.

## Objectives

- Load and preview the raw property and postcode datasets
- Inspect schema, data types, and potential quality issues
- Rename columns for clarity and consistency
- Clean key fields such as `price` and `date` for analysis
- Prepare for postcode enrichment using geospatial metadata
- Document each transformation step with clear rationale and assumptions

## Dataset Overview

- `pp_data_man.parquet`: Property transaction records across Greater Manchester
- `pc_man.parquet`: Postcode metadata including latitude and longitude

## Notes

- All data wrangling is performed using Polars for performance and reproducibility
- This notebook serves as the foundation for analysis and dashboard creation
- Each step is modular and documented to support transparency and future reuse


### Step 1: Load and preview the data

I begin by loading the two datasets
- `pp_data_man.parquet` : This is property transactional records
- `pc_man.parquet` : This data is postcode meta data for referencing geospatial data

In [42]:
import polars as pl

# Define paths
path1 = "../data/pp_data_man.parquet"
path2 = "../data/pc_man.parquet"

# Load datasets
geo_df = pl.read_parquet(path2)
prop_df = pl.read_parquet(path1)

# Preview first few rows
geo_df.head()


pcds,lat,long
str,f64,f64
"""AL1 3PE""",51.755864,-0.327577
"""AL1 4FJ""",51.761598,-0.326075
"""AL1 5UE""",51.751596,-0.325894
"""AL2 1NT""",51.720229,-0.295058
"""AL2 2PJ""",51.721518,-0.337343


In [43]:
prop_df.head()

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4…","""140,000.00""","""2022-02-16 00:…","""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHE…"
"""{2ACACE8D-02B4…","""278,000.00""","""2024-11-29 00:…","""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHE…"
"""{01EB45EF-F1B0…","""345,448.00""","""2022-02-11 00:…","""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STR…","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHE…"
"""{879537F9-FB6B…","""93,000.00""","""2004-09-22 00:…","""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET…","""BURY""","""BURY""","""BURY""","""GREATER MANCHE…"
"""{7011B10A-2B91…","""80,000.00""","""2018-05-03 00:…","""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHE…"


### Step 2: Schema Inspection

After previewing the first few rows of the property sales dataset, it's clear that the Parquet file lacks meaningful column names. All columns are currently labeled as `column_1`, `column_2`, etc., which makes exploration and cleaning difficult until proper names are assigned.

Below, I inspect the schema for both dataframes to understand their structure and prepare for renaming and transformation.

Fortunately, within the briefing document the column names have been given. But good practice to check myself. 


In [44]:
# Schema Inspection

geo_df.schema


OrderedDict([('pcds', String), ('lat', Float64), ('long', Float64)])

In [45]:
prop_df.schema

OrderedDict([('column_1', String),
             ('column_2', String),
             ('column_3', String),
             ('column_4', String),
             ('column_5', String),
             ('column_6', String),
             ('column_7', String),
             ('column_8', String),
             ('column_9', String),
             ('column_10', String),
             ('column_11', String),
             ('column_12', String),
             ('column_13', String),
             ('column_14', String)])

In [46]:
prop_df.head(5)


column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4…","""140,000.00""","""2022-02-16 00:…","""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHE…"
"""{2ACACE8D-02B4…","""278,000.00""","""2024-11-29 00:…","""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHE…"
"""{01EB45EF-F1B0…","""345,448.00""","""2022-02-11 00:…","""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STR…","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHE…"
"""{879537F9-FB6B…","""93,000.00""","""2004-09-22 00:…","""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET…","""BURY""","""BURY""","""BURY""","""GREATER MANCHE…"
"""{7011B10A-2B91…","""80,000.00""","""2018-05-03 00:…","""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHE…"


### Step 3: Assign column names

(A quick note to myself, unlike pandas, polars is immuatable by design. This means that every transformation returns a new data frame which is typically reassigned the same variable. So no need for inpalce=True (pandas))

Based from the project brief, the property sales data set contains 14 named columns. These names will now be assigned to the data frame for clearer exploration and cleaning.

In [47]:
prop_df.columns

['column_1',
 'column_2',
 'column_3',
 'column_4',
 'column_5',
 'column_6',
 'column_7',
 'column_8',
 'column_9',
 'column_10',
 'column_11',
 'column_12',
 'column_13',
 'column_14']

In [48]:
# List of new names from project brief 
column_names = [
    "id", "price", "date", "postcode", "property_type",
    "new", "duration", "paon", "saon", "street",
    "locality", "town_city", "district", "county"
]
# Rename cols using dictionary comprehension
# "column_{i+1}" gets original col name, "column_names[i]" gets the corresponding name from the list provided.
# the result gives "column_1": "id" and so on. 
prop_df = prop_df.rename({f"column_{i+1}": column_names[i] for i in range(14)})

# Returns a list of column names. 
prop_df.columns


['id',
 'price',
 'date',
 'postcode',
 'property_type',
 'new',
 'duration',
 'paon',
 'saon',
 'street',
 'locality',
 'town_city',
 'district',
 'county']

In [49]:
prop_df.head()

id,price,date,postcode,property_type,new,duration,paon,saon,street,locality,town_city,district,county
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4…","""140,000.00""","""2022-02-16 00:…","""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHE…"
"""{2ACACE8D-02B4…","""278,000.00""","""2024-11-29 00:…","""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHE…"
"""{01EB45EF-F1B0…","""345,448.00""","""2022-02-11 00:…","""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STR…","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHE…"
"""{879537F9-FB6B…","""93,000.00""","""2004-09-22 00:…","""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET…","""BURY""","""BURY""","""BURY""","""GREATER MANCHE…"
"""{7011B10A-2B91…","""80,000.00""","""2018-05-03 00:…","""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHE…"


### Step 4: Clean Price column

Right now the Price column is a string data type and it contains both commas and quotes. I want to convert it to a floating point number and remove the  commas. 

In [50]:
# with_columns is how polars updates or removes columns
prop_df = prop_df.with_columns([
    prop_df["price"]                # This selects the price column 
    .str.replace_all(",", "")       # Remove commas
    .str.strip_chars('"')           # Remove surrounding quotes
    .cast(pl.Float64)               # Convert to float
    .alias("price")                 # Keep the column name
])

prop_df.head()

id,price,date,postcode,property_type,new,duration,paon,saon,street,locality,town_city,district,county
str,f64,str,str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4…",140000.0,"""2022-02-16 00:…","""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHE…"
"""{2ACACE8D-02B4…",278000.0,"""2024-11-29 00:…","""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHE…"
"""{01EB45EF-F1B0…",345448.0,"""2022-02-11 00:…","""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STR…","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHE…"
"""{879537F9-FB6B…",93000.0,"""2004-09-22 00:…","""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET…","""BURY""","""BURY""","""BURY""","""GREATER MANCHE…"
"""{7011B10A-2B91…",80000.0,"""2018-05-03 00:…","""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHE…"


### Step 5: Format date column

Currently, the date column is a string with a time stamp suffix. This is not appropriate for time-based analysis
To fix it, I need to convert the string into proper date type. 
This will allow for filtering and time-based aggregation later on down the line.


In [51]:
# Clean the 'date' column:
# - Strip surrounding quotes
# - Parse full datetime string using correct format
# - Convert to Polars Date type (drops time portion)
prop_df = prop_df.with_columns([
    prop_df["date"]
    .str.strip_chars('"')                   # Remove surrounding quotes
    .str.strptime(pl.Date, "%Y-%m-%d %H:%M")  # Parse full datetime string
    .alias("date")                          # Keep the column name
])


### Step 6: Enrich Property Data with Geospatial Coordinates

To support location-based analysis, I will enrich the property sales dataset with latitude and longitude information from the postcode metadata.

Steps:
- Load and preview the postcode dataset (`pc_man.parquet`)
- Standardize the join keys: the property dataset uses `"postcode"` while the postcode dataset uses `"pcds"`
- Rename and clean both columns to ensure a successful join
- Perform a left join to append `lat` and `long` to each property record


In [52]:
geo_df = pl.read_parquet("../data/pc_man.parquet")
geo_df.head()


pcds,lat,long
str,f64,f64
"""AL1 3PE""",51.755864,-0.327577
"""AL1 4FJ""",51.761598,-0.326075
"""AL1 5UE""",51.751596,-0.325894
"""AL2 1NT""",51.720229,-0.295058
"""AL2 2PJ""",51.721518,-0.337343


In [53]:
# Strip quotes from postcode column in prop_df
prop_df = prop_df.with_columns([
    prop_df["postcode"].str.strip_chars('"').alias("postcode")
])

# Rename join key in geo_df to match
geo_df = geo_df.rename({"pcds": "postcode"})

# Join property data with postcode metadata
prop_df = prop_df.join(geo_df, on="postcode", how="left")

# Validating the join to ensure things have joined correctly
prop_df.select(["postcode", "lat", "long"]).head()

prop_df.head()

id,price,date,postcode,property_type,new,duration,paon,saon,street,locality,town_city,district,county,lat,long
str,f64,date,str,str,str,str,str,str,str,str,str,str,str,f64,f64
"""{D93B27B1-CBD4…",140000.0,2022-02-16,"""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHE…",53.477466,-2.091735
"""{2ACACE8D-02B4…",278000.0,2024-11-29,"""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHE…",53.427355,-2.102642
"""{01EB45EF-F1B0…",345448.0,2022-02-11,"""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STR…","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHE…",53.478898,-2.224708
"""{879537F9-FB6B…",93000.0,2004-09-22,"""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET…","""BURY""","""BURY""","""BURY""","""GREATER MANCHE…",53.594024,-2.318071
"""{7011B10A-2B91…",80000.0,2018-05-03,"""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHE…",53.56682,-2.458227


### Step 7: Handling Missing Values

After inspecting the dataframe for missing values, it is clear that 1,265 records are missing latitude and longitude data. Since geospatial analysis is a key part of this project, I have decided to drop these records to ensure consistency and accuracy in location-based insights.



In [54]:
# Check for missing values
prop_df.null_count()

# Drop rows with missing geospatial data by filtering lat and long is not null
prop_df = prop_df.filter(pl.col("lat").is_not_null() & pl.col("long").is_not_null())

### Step 8: Standardising Categorical Fields

To make the dataset more intuitive for stakeholders, I mapped shorthand codes to human-readable labels. This improves clarity in dashboards and removes the need for a legend—for example, replacing `"F"` with `"Flat"` makes the data instantly understandable. It also enhances professionalism and accessibility in visual outputs.

Ultimately, this step is about making the data usable and trustworthy, especially when shared across teams or presented to non-technical audiences.

Initially, I used `map_dict`, but received a deprecation warning. After some research, I found that `replace()` achieves the same result while keeping the pipeline future-proof.


In [55]:
# Define mapping dictionaries
property_type_map = {
    "F": "Flat",
    "S": "Semi Detached",
    "D": "Detached",
    "T": "Terraced",
    "O": "Other (Non Residential)"
}

new_build_map = {
    "Y": "New Build",
    "N": "Old Build"
}

duration_map = {
    "F": "Freehold",
    "L": "Leasehold"
}

# Apply mappings using replace()
prop_df = prop_df.with_columns([
    pl.col("property_type").replace(property_type_map).alias("property_type"),
    pl.col("new").replace(new_build_map).alias("new"),
    pl.col("duration").replace(duration_map).alias("duration")
])

# Validate result
prop_df.select(["property_type", "new", "duration"]).head()


property_type,new,duration
str,str,str
"""Terraced""","""Old Build""","""Freehold"""
"""Semi Detached""","""Old Build""","""Freehold"""
"""Flat""","""New Build""","""Leasehold"""
"""Terraced""","""Old Build""","""Leasehold"""
"""Terraced""","""Old Build""","""Freehold"""


### Step 9: Creating Derived Columns

This step is important because it makes querying the data faster, cleaner, and more reproducible. It also aligns directly with the challenge questions, many of which require filtering by year, aggregating by postal sector, or identifying trends over time.

By extracting key components like `year` from `date` and `postal_sector` from `postcode`, I can simplify filtering, grouping, and visualisation. These derived columns also improve performance and readability in dashboards, making the data more accessible to stakeholders.

Examples include:
- `year`: Enables time-based grouping (e.g. sales after 2010, trends since 1996)
- `postal_sector`: Supports spatial aggregation (e.g. ranking sectors like "M1 3" or "M3 6")

These transformations help turn raw data into actionable dimensions for analysis.


In [56]:
# Extract year from date using `dt.year()` method
prop_df = prop_df.with_columns([
    pl.col("date").dt.year().alias("year")
])

# Preview result
prop_df.select(["date", "year"]).head()


date,year
date,i32
2022-02-16,2022
2024-11-29,2024
2022-02-11,2022
2004-09-22,2004
2018-05-03,2018


In [57]:
# Derive postal_sector from postcode
# Select postcode col
# Applied a regex to each string that extracts the postal sector, first part of post code + first digit of the second part
# The number 1 refers to the first capturing group of the expression
# Alias renames the result to postal_sector so it becomes a new col in the dataframe
prop_df = prop_df.with_columns([
    pl.col("postcode").str.extract(r"^([A-Z]+\d+\s\d)", 1).alias("postal_sector")
])

# Preview result
prop_df.select(["postcode", "postal_sector"]).head()

postcode,postal_sector
str,str
"""SK16 4DT""","""SK16 4"""
"""SK6 1QW""","""SK6 1"""
"""M1 2EY""","""M1 2"""
"""BL8 2RR""","""BL8 2"""
"""BL3 4HE""","""BL3 4"""


### Step 10: Checking Data Ranges

To check for outliers and ensure the dataset falls within the expected scope (1996–2024), I validated the minimum and maximum values for `date`. This helps identify any suspicious entries, such as dates outside the challenge range.

In [59]:
prop_df.select([
    pl.col("date").min().alias("min_date"),
    pl.col("date").max().alias("max_date")
])


min_date,max_date
date,date
1995-01-01,2025-07-31


### Step 11: Filtering by Scope and Timeframe

To align with the challenge requirements, I filtered the dataset to include only:

- Sales between **January 1996 and December 2024**
- Properties within **Greater Manchester**

This ensures all subsequent analysis is relevant and scoped correctly.

I had to import `date` from Python’s `datetime` module because Polars requires native datetime objects for comparisons.

In [63]:
# for safe date comparisons
from datetime import date
# Filtered data set to include project scope
prop_df = prop_df.filter(
    (pl.col("date") >= date(1996, 1, 1)) & # Keep rows on and after 1996-01-01
    (pl.col("date") <= date(2024, 12, 31)) # The same but on and before 2024-12-31
)
# validate by checking min and max dates to ensure within project scope
prop_df.select([
    pl.col("date").min().alias("min_date"), # Earliest sale
    pl.col("date").max().alias("max_date") # Latest sale
])


min_date,max_date
date,date
1996-01-01,2024-12-26


In [64]:
# This code snippet does the same but for county, ensuring only the greater manchester area is included 
prop_df = prop_df.filter(
    pl.col("county") == "GREATER MANCHESTER"
)
